In [1]:
import numpy as np
import pandas as pd
import scipy.stats as si
from scipy.stats import norm
from math import log, sqrt, exp
import yfinance as yf
from datetime import datetime
import datetime as dt

In [2]:
#function to calculate d1
def d1(S, K, T, r, sigma):
    D1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return D1

In [3]:
#function to calculate d2
def d2(S, K, T, r, sigma):
    dd1 = d1(S, K, T, r, sigma)
    D2 = dd1 - sigma * np.sqrt(T)
    return D2

In [4]:
def black_scholes(S, K, T, r, option_type):
    if option_type == "call":
        # Calculate call option price
        call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        return call_price
    elif option_type == "put":
        # Calculate put option price
        put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        return put_price
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

In [5]:
#function to calculate delta
def calc_delta(S, K, T, r, sigma, option_type):
    D1 = d1(S, K, T, r, sigma)
    if option_type == 'call':
        return si.norm.cdf(D1)
    elif option_type == 'put':
        return si.norm.cdf(D1) - 1

In [6]:
#function to calculate gamma
def calc_gamma(S, K, T, r, sigma):
    D1 = d1(S, K, T, r, sigma)
    return si.norm.pdf(D1) / (S * sigma * np.sqrt(T))

In [7]:
#function to calculate theta
def calc_theta(S, K, T, r, sigma, option_type='call'):
    D1 = d1(S, K, T, r, sigma)
    D2 = d2(S, K, T, r, sigma)
    term1 = -(S * si.norm.pdf(D1) * sigma) / (2 * np.sqrt(T))

    if option_type == 'call':
        term2 = r * K * np.exp(-r * T) * si.norm.cdf(D2)
        return term1 - term2
    elif option_type == 'put':
        term2 = r * K * np.exp(-r * T) * si.norm.cdf(-D2)
        return term1 + term2

In [8]:
#function to calculate vega
def calc_vega(S, K, T, r, sigma):
    D1 = d1(S, K, T, r, sigma)
    return S * si.norm.pdf(D1) * np.sqrt(T)

In [9]:
def get_stock_data(ticker):
    try:
        # Get stock data
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1y")

        if hist.empty:
            raise ValueError(f"No data found for ticker {ticker}")

        # Calculating daily returns and volatility
        daily_returns = hist['Close'].pct_change()
        annual_volatility = daily_returns.std() * np.sqrt(252)

        current_price = hist['Close'].iloc[-1]
        risk_free_rate = get_risk_free_rate()

        return {
            'price': current_price,
            'volatility': annual_volatility,
            'risk_free_rate': risk_free_rate
        }
    except Exception as e:
        raise ValueError(f"Error fetching data for {ticker}: {str(e)}")

In [10]:
def get_risk_free_rate():
    try:
        treasury = yf.Ticker("^TNX")
        current_yield = treasury.history(period="1d")['Close'].iloc[-1] / 100
        return current_yield
    except:
        print("Could not fetch risk-free rate. Using default value of 0.03 (3%)")
        return 0.03

In [11]:
def calculate_time_to_expiry(expiry_date):
    try:
        if isinstance(expiry_date, str):
            expiry = datetime.strptime(expiry_date, '%Y-%m-%d')
        else:
            expiry = expiry_date

        today = datetime.now()
        days_to_expiry = (expiry - today).days

        if days_to_expiry < 0:
            raise ValueError("Expiration date must be in the future")

        return days_to_expiry / 365
    except ValueError as e:
        raise ValueError(f"Invalid date format. Please use YYYY-MM-DD format. Error: {str(e)}")

In [ ]:
def main():
    print("This is a Black-Scholes Option Pricing Model")

    while True:
        try:
            # Get stock ticker and fetch data
            ticker = input("\nEnter stock ticker symbol (e.g., AAPL): ").upper()
            print(f"\nFetching data for {ticker}...")

            stock_data = get_stock_data(ticker)
            S = stock_data['price']
            sigma = stock_data['volatility']
            r = stock_data['risk_free_rate']

            print(f"\nCurrent stock price: ${S:.2f}")
            print(f"Historical volatility: {sigma:.2%}")
            print(f"Risk-free rate: {r:.2%}")

            # Get other parameters
            K = float(input("\nEnter strike price: "))
            expiry_date = input("Enter expiration date (YYYY-MM-DD): ")
            T = calculate_time_to_expiry(expiry_date)
            print(f"Time to expiry: {T:.3f} years")

            option_type = input("Enter option type (call/put): ").lower()

            if option_type not in ['call', 'put']:
                print("Invalid option type. Please enter 'call' or 'put'.")
                continue

            # Calculate results for both call and put
            results = {
                'call': black_scholes(S, K, T, r, sigma, 'call'),
                'put': black_scholes(S, K, T, r, sigma, 'put')
            }

            # Display results in a table
            headers = ['Metric', 'Call Option', 'Put Option']
            table_data = []
            for metric in ['price', 'delta', 'gamma', 'vega', 'theta']:
                row = [
                    metric.capitalize(),
                    f"{results['call'][metric]:.4f}",
                    f"{results['put'][metric]:.4f}"
                ]
                table_data.append(row)

            print("\nResults:")

            # Ask if user wants to continue
            if input("\nCalculate another option? (y/n): ").lower() != 'y':
                break

        except ValueError as e:
            print(f"Error: {str(e)}")
            continue
        except Exception as e:
            print(f"An unexpected error occurred: {str(e)}")
            continue

if __name__ == "__main__":
    main()

This is a Black-Scholes Option Pricing Model

Enter stock ticker symbol (e.g., AAPL): AAPL

Fetching data for AAPL...

Current stock price: $226.80
Historical volatility: 22.45%
Risk-free rate: 3.98%

Enter strike price: 230
Enter expiration date (YYYY-MM-DD): 2024-11-15
Time to expiry: 0.104 years
Enter option type (call/put): call
An unexpected error occurred: black_scholes() takes 5 positional arguments but 6 were given
